# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# load csv output_data_file from WeatherPy exercise
os.chdir('..')
data_df = pd.read_csv(r'WeatherPy\output_data\cities.csv')
data_df = data_df.dropna()
data_df.head()


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,bac lieu,30.0,VN,1.585006e+09,83.0,9.29,105.72,77.09,3.53
1,leningradskiy,100.0,RU,1.585006e+09,75.0,69.38,178.42,27.23,4.63
2,porvoo,75.0,FI,1.585006e+09,80.0,60.39,25.67,36.00,13.87
4,nikolskoye,0.0,RU,1.585006e+09,54.0,59.70,30.79,28.40,4.47
5,tulun,100.0,RU,1.585006e+09,90.0,54.56,100.58,15.35,7.72


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Pull latitude and longitud values into new dataframe 
locations = data_df[["Lat", "Lng"]].astype(float)
# Pull humidity values into new dataframe
humidity = data_df["Humidity"].astype(float)

# Create a humidity heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
# Add heatmap layer to map
fig.add_layer(heat_layer)

fig

# Image downloaded and saved in VacationPy folder "humidity_heatmap"

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Create new dataframe with 70 < Max Temp < 80, Wind Speed < 10mph, and zero cloudiness
ideal_data_df = data_df[data_df["Max Temp"] > 70]
ideal_data_df = ideal_data_df[ideal_data_df["Max Temp"] < 80]
ideal_data_df = ideal_data_df[ideal_data_df["Wind Speed"] < 10]
ideal_data_df = ideal_data_df[ideal_data_df["Cloudiness"] == 0]

ideal_data_df = ideal_data_df.dropna()

ideal_data_df


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
157,port hedland,0.0,AU,1.585006e+09,74.0,-20.32,118.57,78.80,8.05
329,santa maria,0.0,BR,1.585006e+09,56.0,-29.68,-53.81,73.67,6.29
365,sompeta,0.0,IN,1.585006e+09,72.0,18.93,84.60,72.59,3.44
373,lingao,0.0,CN,1.585006e+09,88.0,19.91,109.69,73.13,9.93
627,gualeguay,0.0,AR,1.585006e+09,44.0,-33.14,-59.31,70.97,9.37


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create new dataframe named hotel_df
hotel_df = ideal_data_df
# Create header "Hotel Name"
hotel_df["Hotel Name"]=""
# Reset index
hotel_df = hotel_df.reset_index()
# Drop old index numbers
hotel_df = hotel_df.drop(columns=["index"])
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,port hedland,0.0,AU,1.585006e+09,74.0,-20.32,118.57,78.80,8.05,
1,santa maria,0.0,BR,1.585006e+09,56.0,-29.68,-53.81,73.67,6.29,
2,sompeta,0.0,IN,1.585006e+09,72.0,18.93,84.60,72.59,3.44,
3,lingao,0.0,CN,1.585006e+09,88.0,19.91,109.69,73.13,9.93,
4,gualeguay,0.0,AR,1.585006e+09,44.0,-33.14,-59.31,70.97,9.37,


In [7]:
# Create counter and set to zero
row_count = 0

#list to collect hotels
hotels = []

# Use Google Places API to find hotel names
for cities in hotel_df["City"]:
    # Use lat and lng to find hotels within target radius of 5000m
    target_coordinates = str(hotel_df['Lat'][row_count])+' , '+str(hotel_df['Lng'][row_count])
    row_count += 1 
    target_radius = 5000
    target_type = 'lodging'
    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key}
    
    # Google places url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    try:
        # Call google places and store hotel names
        response = requests.get(base_url, params=params)
        hotel_results = response.json()
        hotels.append(hotel_results['results'][0]['name'])                
        # Do not store NaN values
    except IndexError:
        hotels.append(np.nan)

# add hotel names to dataframe
hotel_df['Hotel Name'] = hotels
hotel_df
    

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,port hedland,0.0,AU,1.585006e+09,74.0,-20.32,118.57,78.80,8.05,The Esplanade Hotel
1,santa maria,0.0,BR,1.585006e+09,56.0,-29.68,-53.81,73.67,6.29,Hotel Morotin
2,sompeta,0.0,IN,1.585006e+09,72.0,18.93,84.60,72.59,3.44,V.B.R A/C Lodge And Function Hall
3,lingao,0.0,CN,1.585006e+09,88.0,19.91,109.69,73.13,9.93,Lingao Ditai Inn
4,gualeguay,0.0,AR,1.585006e+09,44.0,-33.14,-59.31,70.97,9.37,Cabañas Nuestro Refugio


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name (hotel_df)
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add markers to identfy hotel locations on map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display Map
fig
# Image downloaded and saved in VacationPy folder "hotel_map"

Figure(layout=FigureLayout(height='420px'))